![clarifai_logo](./images/clarifai_logo.png)

# Table of Content

### * [Importing the libraries](#libraries)


### * [What are API keys](#api_keys)


### * [Examples of Working with Public Models](#public_models)
    
####                 1.  [General Model](#general_model)
####                 2.  [Demographics Workflow](#demographics)
####                 2.  [Custom Model Predictions](#custom_model)


### * [Visual Search](#visual_search)

### * [Calling a Public Detection Model - Apparel Detection ](#apparel-detection)

<a id ='libraries'></a>
# Let's start by importing the required libaries 

In [1]:
### Clarifai libraries

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc,service_pb2,resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2
import numpy as np



In [2]:
# Create a stub which will be used to interact with the API 
stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())


<a id = 'api_keys'></a>
# API Keys

* We interact with the Clarifai API through an API Key which is used to make calls to the API. 


* We can get an API by signing up for a Clarifai account and then getting an API key by creating an application. You can read about that at this link: https://docs.clarifai.com/clarifai-basics/authentication/app-specific-api-keys


* You can use the API key to make calls to the public models whose list can be found at the link below 
https://www.clarifai.com/developers/model-gallery


* Once you have generated an API key please copy/paste it in the following declartaion and replace it with *YOUR_API_KEY*


In [12]:
# Setting up an API connection with an API key
# This is how you authenticate.
metadata = (('authorization', 'Key YOUR_API_KEY'),)



<a id ='public_models'></a>
# Public model examples

* We have public models which can be used to answer pressing business related use-cases. In fact, many of our clients use our public models to move their business forward.


* The great thing about public models is that they come built-in with powerful insights which you can get up and runnning with minimal effort as a developer. 


* You can just swap out the model in the calls and make calls to the differernt models. You can get the model ids for each model in their appropriate sections in the Model Gallery or by going to the Model Model in an application on Clarifai's Portal. (https://www.clarifai.com/developers/model-gallery)


* For the inputs we will provide for prediction, it needs to either be a publically accessible URL or an image/video/text from your computer. In this Notebook I will use publically accessible URLs but you can find details about how to add images from your local computer here (https://docs.clarifai.com/api-guide/data/create-get-update-delete#add-an-input-using-bytes)

<a id = 'general_model'></a>
## General Model 

### We will be looking at the predictions from the General Model for this image

![Input image for General Model](https://portal.clarifai.com/cms-assets/20180320225539/general-008.jpg)


In [4]:
#Input URL : https://portal.clarifai.com/cms-assets/20180320225539/general-008.jpg

request = service_pb2.PostModelOutputsRequest(
    # This is the model ID of a publicly available General model. You may use any other public or custom model ID.
    model_id='aaa03c23b3724a16a56b629203edc62c',
    inputs=[
      resources_pb2.Input(data=resources_pb2.Data(image=resources_pb2.Image(url='https://portal.clarifai.com/cms-assets/20180320225539/general-008.jpg')))
    ])
response = stub.PostModelOutputs(request, metadata=metadata)

if response.status.code != status_code_pb2.SUCCESS:
    raise Exception("Request failed, status code: " + str(response.status.code))
else:
    print('Prediction made successfully')



Prediction made successfully


**We have made a prediction and now let's take a look at the JSON object returned in the variable *response* to see what the raw response looks like.**


In [5]:
response

status {
  code: SUCCESS
  description: "Ok"
  req_id: "46bede1b5d230f2f558ca1b99fa72783"
}
outputs {
  id: "e1acd80fda8348f9810457a0d05fcf7a"
  status {
    code: SUCCESS
    description: "Ok"
  }
  created_at {
    seconds: 1655485255
    nanos: 86103424
  }
  model {
    id: "general-image-recognition"
    name: "Image Recognition"
    created_at {
      seconds: 1457543499
      nanos: 608845000
    }
    app_id: "main"
    output_info {
      output_config {
      }
      message: "Show output_info with: GET /models/{model_id}/output_info"
      fields_map {
        fields {
          key: "concepts"
          value {
            string_value: "softmax"
          }
        }
      }
    }
    model_version {
      id: "aa7f35c01e0642fda5cf400f543e7c40"
      created_at {
        seconds: 1520370624
        nanos: 454834000
      }
      status {
        code: MODEL_TRAINED
        description: "Model is trained and ready"
      }
      visibility {
        gettable: PUBLIC
      }

**There is a lot of information here. Let's just take a look at the predicted values by running the following block of code**

In [6]:
for concept in response.outputs[0].data.concepts:
    print('%12s: %.2f' % (concept.name, concept.value))

         dog: 1.00
       grass: 0.98
      mammal: 0.98
      canine: 0.97
        cute: 0.97
         pet: 0.96
       puppy: 0.96
      animal: 0.95
   no person: 0.93
       whelp: 0.89
       field: 0.88
         fur: 0.87
   retriever: 0.87
    portrait: 0.86
       funny: 0.81
    adorable: 0.79
      little: 0.79
      tongue: 0.78
    hayfield: 0.76
golden retriever: 0.75


<a id = 'demographics'></a>
## Demographics Workflow 

* In Clarifai's Platform you can chain together different models in a workflow and get predictions from all of them for an input at once. That is what the Demographics workflow does.


* In the demographics workflow we chain together three models 
    1. Age 
    2. Multi-cultural appearence 
    3. Gender appearence 
    
    
* We will send an input for prediction to the Demographics Workflow which will give us predictions for all those models. To do this, we will make a prediction to the Workflow predict endpoint which is same for public as well as custom workflows. You can read more about Workflow predict here (https://docs.clarifai.com/api-guide/workflows/common-workflows/workflow-predict)


### We will be looking at the predictions of the Demographics workflow for this image 

![Image for Demographics workflow](https://images.unsplash.com/photo-1544005313-94ddf0286df2?ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=634&q=80)

In [7]:
demo_response = stub.PostWorkflowResults(
    service_pb2.PostWorkflowResultsRequest(
        workflow_id="Demographics",
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url="https://images.unsplash.com/photo-1544005313-94ddf0286df2?ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=634&q=80"
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if demo_response.status.code != status_code_pb2.SUCCESS:
    raise Exception("Post workflow results failed, status: " + demo_response.status.description)
else:
    print('Prediction made successfully')
    

Prediction made successfully


**Let's take a look at the response object**

In [8]:
demo_response

status {
  code: SUCCESS
  description: "Ok"
  req_id: "b4a92b2ff3497eb9ac1cb9389e406323"
}
workflow {
  id: "Demographics"
  app_id: "main"
  created_at {
    seconds: 1598622565
    nanos: 390923000
  }
  metadata {
    fields {
      key: "icon"
      value {
        struct_value {
          fields {
            key: "url"
            value {
              string_value: "https://s3.amazonaws.com/samples.clarifai.com/featured-models/workflow-demographics-group-of-people.png"
            }
          }
        }
      }
    }
    fields {
      key: "presetInputs"
      value {
        list_value {
          values {
            struct_value {
              fields {
                key: "type"
                value {
                  string_value: "image"
                }
              }
              fields {
                key: "url"
                value {
                  string_value: "https://s3.amazonaws.com/samples.clarifai.com/featured-models/workflow-demographics-old-blac

**Let's take a look at only the results since this is a lot of information. We will start by looking at the predictions from the multi-cultural appearence model**

In [9]:
mc_response = demo_response.results[0].outputs[2].data.regions[0].data.concepts # multicultural appearence outputs 

for i in range(len(mc_response)):
    print(mc_response[i].name,':', np.around(mc_response[i].value,4))

White : 0.4822
Latino_Hispanic : 0.36
Middle Eastern : 0.1211
Indian : 0.0227
Black : 0.0115
East Asian : 0.0014
Southeast Asian : 0.0011


**Let's now take a look at the values being predicted by the gender appearence model.**

In [10]:
gender_response = demo_response.results[0].outputs[3].data.regions[0].data.concepts
# gender_response

for i in range(len(gender_response)):
    print(gender_response[i].name,':', np.around(gender_response[i].value,4))

Feminine : 1.0
Masculine : 0.0


**Let's take a look at the Age Appearence where we get approximate predictions of the input from this model**

In [11]:
age_appearence = demo_response.results[0].outputs[4].data.regions[0].data.concepts
# age_appearence

for i in range(len(age_appearence)):
    print(age_appearence[i].name,':', np.around(age_appearence[i].value,4))

20-29 : 0.8755
10-19 : 0.0671
30-39 : 0.057
40-49 : 0.0004
3-9 : 0.0
50-59 : 0.0
0-2 : 0.0
60-69 : 0.0
more than 70 : 0.0


<a id ='custom_model'></a>
## Custom Model predictions 

* With Clarifai you can create custom models and train them using our platform and then make predictions to it. 

* For Custom model predictions to work, the API key you provide will need to be from the application where the custom model is. 


### You can read more about the custom model creation and predictions here 

* Via Portal: https://docs.clarifai.com/portal-guide/model/pcustom-model-walkthrough

* Via the API : https://docs.clarifai.com/api-guide/model/custom-model-walkthrough


I have a model where I trained it to identify four different kind of vehicles, I will be making predicitons to it to work this code. You will need to put the right API key in the following code block and give it the name of the model you created which is the model id and it should work as well 


In [97]:
# Setting up an API connection with an API key
# The API key needs to be from the application where the model was created. 

metadata = (('authorization', 'Key YOUR_API_KEY'),)

custom_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        model_id="vehicle-finder", # replace the id here with your model id 
#         version_id="{YOUR_MODEL_VERSION_ID}",  # This is optional. Defaults to the latest model version.
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url="https://samples.clarifai.com/metro-north.jpg"
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if custom_response.status.code != status_code_pb2.SUCCESS:
    raise Exception("Post model outputs failed, status: " + custom_response.status.description)
else:
    print('prediction made successfully')


prediction made successfully


In [91]:
# Since we have one input, one output will exist here.
output = custom_response.outputs[0].data.regions[0].data.concepts

output


[id: "bicycles"
name: "bicycles"
value: 0.12244516
app_id: "vehicle-labeling"
, id: "buses"
name: "buses"
value: 4.7368027e-05
app_id: "vehicle-labeling"
, id: "cars"
name: "cars"
value: 1.089022e-06
app_id: "vehicle-labeling"
, id: "motorcycle"
name: "motorcycle"
value: 2.679583e-10
app_id: "vehicle-labeling"
]

In [92]:
# print the custom predicted values

In [94]:
for i in range(len(output)):
    print(output[i].name,':', np.around(output[i].value,4))

bicycles : 0.1224
buses : 0.0
cars : 0.0
motorcycle : 0.0


<a id = 'visual_search'></a>
## Visual Search 

**What is visual search?**

Visual search is when you search your catalog by visual similarity. Let's say that I run retail company which sells clothes and I would like to introduce a visual similarity funcitonality for my customers. I want them to be able to take images of clothes they like and search within our catalog to find similar clothes which they might like. 

Using Clarifai you can easily implement this use-case. You could upload images of your catalog to an app using our **Add Inputs** endpoint in the API which you can read here (https://docs.clarifai.com/api-guide/data/create-get-update-delete#add-inputs). So now when you have to search against your entire catalog you can do it in just 1 API call, by asking for a visual search request. 

This would take the image you just submitted and re-index (sort) your entire inputs based on how similar they are to the image you just provided. You could refine this use-case by adding metadata to your iputs when you add them to an application. You could add different things to metadata, such as the product URL to redirect to, name of the product or any number of things. You can add inputs with metadata as shown in this documentation. (https://docs.clarifai.com/api-guide/data/create-get-update-delete#add-inputs-with-custom-metadata)


**You would use the code shown in the following docs to make a visual search call** 
(https://docs.clarifai.com/api-guide/search/rank#search-by-image-url)

I have an app where I have images stored for a retail client use-case. I am going to make a visual search request to this app with an image of a pair of glasses I like to see if I can find anything that is similar to what I am looking for. The glasses I am looking for and I want to see what the catalog has that is similar to what i like 

![](https://images-na.ssl-images-amazon.com/images/I/71AIdaQLMmL._AC_UL1500_.jpg)


In [5]:
# The API key here will be the API key from the app where you have added your inputs to 
metadata = (('authorization', 'Key YOUR_API_KEY'),)

search_response = stub.PostAnnotationsSearches(
    service_pb2.PostAnnotationsSearchesRequest(
        searches = [
            resources_pb2.Search(
                query=resources_pb2.Query(
                    ranks=[
                        resources_pb2.Rank(
                            annotation=resources_pb2.Annotation(
                                data=resources_pb2.Data(
                                    image=resources_pb2.Image(
                                        url="https://images-na.ssl-images-amazon.com/images/I/71AIdaQLMmL._AC_UL1500_.jpg"
                                    )
                                )
                            )
                        )
                    ]
                )
            )
        ]
    ),
    metadata=metadata
)



### Let's take a look at the scores returned

In [6]:
if search_response.status.code != status_code_pb2.SUCCESS:
    raise Exception("Post searches failed, status: " + search_response.status.description)

print("Search result:")
for hit in search_response.hits:
    print("\tScore %.2f for annotation: %s off input: %s" % (hit.score, hit.annotation.id, hit.input.id))


Search result:
	Score 0.96 for annotation: a046ca1ead4f4cd98ffcb44797734c46 off input: 1002701328
	Score 0.96 for annotation: 9b0faa4c553f463b9922c7a8ef8dcaf3 off input: 1002701312
	Score 0.96 for annotation: ee3603df9669402089497812ff91474c off input: 1001853876
	Score 0.96 for annotation: 7ebc1decfbf241d4aa20da8592ee3c50 off input: 1001852888
	Score 0.96 for annotation: 1ecfee6e930f491aa858529b266af471 off input: 1001863122
	Score 0.96 for annotation: 938a35e694094b5f9a2712551fbc5a42 off input: 50223913
	Score 0.96 for annotation: 64c74ea40c614c9097c13b670d2b7f6f off input: 1001863570
	Score 0.96 for annotation: dc2a2abe161c40668d1f6cedead68d2d off input: 1001854000
	Score 0.96 for annotation: c3f379148f834448a2997087cd820b00 off input: 1001853080
	Score 0.96 for annotation: 514f298c104641259c2e93ba9f6e8251 off input: 1001853104
	Score 0.96 for annotation: b74476dbdf7b45a4b88f24abeda198cc off input: 1000608413
	Score 0.96 for annotation: 927598e7a65548418bb080671e47ea9f off input: 10

### This is interesting and all but let's look at the raw data returned and see if we can come up with more interesting way of using this data. 

The following response shows the raw response where the images are being returned with their metadata which we added at the time of upload 

In [8]:
search_response.hits

[score: 0.96423835
input {
  id: "1002701328"
  data {
    image {
      url: "https://images.lowes.com/product/converted/731919/731919860361.jpg"
      hosted {
        prefix: "https://s3.amazonaws.com/clarifai-api/img3/prod"
        suffix: "3617f25f9ee5463497e7abf350091b93/7461e5b312e231f5541da62f27854bb6"
        sizes: "orig"
        sizes: "tiny"
        sizes: "small"
        sizes: "large"
      }
    }
    metadata {
      fields {
        key: "cat_entry_id"
        value {
          string_value: "1002701328"
        }
      }
      fields {
        key: "catalog_name"
        value {
          string_value: "Sunglasses & Glasses"
        }
      }
      fields {
        key: "catalog_path"
        value {
          string_value: "Tools$Safety & Workwear$Sunglasses & Glasses"
        }
      }
      fields {
        key: "link_url"
        value {
          string_value: "https://www.lowes.com/pd/Magnifeye-Magnifeye-Retro-Black-1-25/1002701328"
        }
      }
      field

### The metadata added to an image in Portal looks like as follows 

![](./images/visual_search_metadata.png)

### Let's look at one returned response to what is actually being returned. We will look at the top returned result. 

In [26]:

metadata = search_response.hits[0].input.data.metadata.fields

print('score :',search_response.hits[0].score,'\n\n')
print('Now printing metadata \n\n')

for k, v in metadata.items():
    print(k,v)


score : 0.9642383456230164 


Now printing metadata 


catalog_path string_value: "Tools$Safety & Workwear$Sunglasses & Glasses"

catalog_name string_value: "Sunglasses & Glasses"

cat_entry_id string_value: "1002701328"

link_url string_value: "https://www.lowes.com/pd/Magnifeye-Magnifeye-Retro-Black-1-25/1002701328"

product_name string_value: "Magnifeye Magnifeye Retro Black 1.25"



### In the above response we can see there being URL returned which we can use to redirect a user to a product that matches what they are looking for baed on thier similarity

<a id='apparel-detection'></a>
## Apparel Detection

**Using Clarifai to answer retail apparel use-cases:**

You can use Clarifai's [Apparel Detection Model](https://www.clarifai.com/models/apparel-detection) to answer real-world fashion industry related use cases. The Model looks at inputs(iamges/videos) and in each input detects different clothing/accessory items by putting bounding boxes around them. 

This is what the Apparel model's output would look like ![apparel_detect_image](./images/apparel_2.png)


You can make a prediction to it, the same way you would make prediction to a custom model. The API endpoint for custom model predictions works by you giving it a model id. In case of a custom model the model is is the name you give it. However, you could provide that endpoint the model id for any other model (General Model. Face Model, Racial appearence model and it will work that way as well. So we are just going to give that endpoint the model id for the Apparel detection model as well as an image to predict on. 

**The model id for the Apparel Detection model is *72c523807f93e18b431676fb9a58e6ad***

We will be running a prediction on the following image. The Apparel Detection model will tell us what items of apparel are in there as well as the bounding boxes where they are 

![appare-detect-test](https://images.pexels.com/photos/1075776/pexels-photo-1075776.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260)



In [16]:
# Setting up an API connection with an API key
# The API key needs to be from the application where the model was created. 

metadata = (('authorization', 'Key YOUR_API_KEY'),)

apparel_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        model_id="72c523807f93e18b431676fb9a58e6ad", #model id for apparel detection model 
#         version_id="{YOUR_MODEL_VERSION_ID}",  # This is optional. Defaults to the latest model version.
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url="https://images.pexels.com/photos/1075776/pexels-photo-1075776.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260"
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if apparel_response.status.code != status_code_pb2.SUCCESS:
    raise Exception("Post model outputs failed, status: " + apparel_response.status.description)
else:
    print('prediction made successfully')


prediction made successfully


In [24]:
apparel_regions = apparel_response.outputs[0].data.regions
print('There detected items of clothing/accessories are: ', len(apparel_regions),'\n\n')


There detected items of clothing/accessories are:  13 




In [25]:
apparel_regions

[id: "6rj0elzvkuft"
region_info {
  bounding_box {
    top_row: 0.7731218
    left_col: 0.23859031
    bottom_row: 0.86608905
    right_col: 0.3198795
  }
}
data {
  concepts {
    id: "ai_6ZGj27W6"
    name: "shoes"
    value: 0.9998909
    app_id: "main"
  }
}
value: 0.9998909
, id: "lrj9q3oh4ama"
region_info {
  bounding_box {
    top_row: 0.35927078
    left_col: 0.22859108
    bottom_row: 0.56041604
    right_col: 0.5675377
  }
}
data {
  concepts {
    id: "ai_nrTL5JlX"
    name: "top"
    value: 0.9998543
    app_id: "main"
  }
}
value: 0.9998543
, id: "7hoaeihh3bnu"
region_info {
  bounding_box {
    top_row: 0.80863893
    left_col: 0.5229257
    bottom_row: 0.91169196
    right_col: 0.6447445
  }
}
data {
  concepts {
    id: "ai_6ZGj27W6"
    name: "shoes"
    value: 0.99953973
    app_id: "main"
  }
}
value: 0.99953973
, id: "g2v1x2g3ebdr"
region_info {
  bounding_box {
    top_row: 0.47035095
    left_col: 0.28079987
    bottom_row: 0.8009345
    right_col: 0.637596
  }
}


**In the above output we can see the number of items that were detected in the image. These items are not only returned with where they are but also where they are in the image which we can use to see where in the image that item was detected. Let's take a look at one items that was detected.**


In [30]:
apparel_regions[np.random.randint(0,len(apparel_regions))]

id: "px7kdw31vn4p"
region_info {
  bounding_box {
    top_row: 0.5579039
    left_col: 0.5728075
    bottom_row: 0.58921874
    right_col: 0.6122371
  }
}
data {
  concepts {
    id: "ai_ckdxd4DP"
    name: "bracelet"
    value: 0.99107575
    app_id: "main"
  }
}
value: 0.99107575

**Let's take a look at what that image would look like in our Portal when we predict on this image.** 

![apparel-demo-final](./images/apparel_demo.png)